## Age Detection

In [42]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from random import shuffle
import os
from keras.models import Model, Sequential, load_model
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, ZeroPadding2D
from keras import applications
from keras.utils import plot_model, to_categorical
from keras.applications import VGG16, ResNet50
from keras.optimizers import RMSprop,Adam 
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import cv2


In [54]:
filenames = os.listdir("train")
shuffle(filenames)
age=[]  #Children (1-14) CLASS 0;Youth (14-25) CLASS 1;ADULTS (25-40) CLASS 2;Middle age (40-60) CLASS 3
         #Very Old (>60) CLASS 4
gender=[] #0=male ;1=female

for filename in filenames:
    label_1=filename.split('_')[0]
    label_1 = int(label_1)
    if label_1 <= 14:
        age.append(str(0))
    if (label_1>14) and (label_1<=25):
        age.append(str(1))
    if (label_1>25) and (label_1<40):
        age.append(str(2))
    if (label_1>=40) and (label_1<60):
        age.append(str(3))
    if label_1>=60:
        age.append(str(4))
   
for filename in filenames:
    label_2=filename.split('_')[1]
    label_2 = int(label_2)
    if label_2 == 0:
        gender.append(str(0))
    if label_2 == 1:
        gender.append(str(1))    

In [55]:
df=pd.DataFrame({
    'filename':filenames,
    'age':age,
    'gender':gender
})
df.head()

,filename,age,gender
0,23_0_3_20170119164007795.jpg.chip.jpg,1,0
1,5_0_3_20161220220614792.jpg.chip.jpg,0,0
2,57_1_0_20170109132244248.jpg.chip.jpg,3,1
3,24_1_0_20170104022800709.jpg.chip.jpg,1,1
4,48_0_0_20170104182220669.jpg.chip.jpg,3,0


In [86]:
img_row = 32
img_col = 32
epochs = 30
batch_size = 128

In [87]:
pre_trained_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_row,img_col,3)) #taking pretrained weights;top layer chopped off as i am going to use it of my own;
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False  #freeze upto 15th layer only of vgg16

for layer in pre_trained_model.layers[15:]:
    layer.trainable = False  #if true fine tuning vgg16 starts from this layer

In [88]:
top_model = pre_trained_model.output #output of vgg16 last layer taken as input here
top_model = Flatten()(top_model)
layer_1 = Dense(1024,activation='relu')(top_model)
layer_2= Dense(1024,activation='relu')(top_model)
#layer_1 = Dense(128,activation='relu')(layer_1)
layer_1 = Dropout(0.5)(layer_1)
layer_2 = Dropout(0.5)(layer_2)
out_1 = Dense(5,activation='softmax',name='out_1')(layer_1) #for age
out_2 = Dense(1,activation='sigmoid',name='out_2')(layer_2) #for gender




In [89]:
model = Model(inputs = pre_trained_model.input ,outputs=out_1) #use out_1 for age and out_2 for gender

In [78]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [61]:
train_df, validate_df = train_test_split(df, test_size=0.1)
train_df = train_df.reset_index()                #arguement ta has to be a pd.dataframe,np.array or list
validate_df = validate_df.reset_index() 
# validate_df = validate_df.sample(n=100).reset_index() # use for fast testing code purpose
# train_df = train_df.sample(n=1800).reset_index() # use for fast testing code purpose

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

print(total_train)
print(total_validate)

21337
2371


In [79]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "train", 
    x_col='filename',
    y_col='age',
    class_mode='categorical',                    #'multi_output'],
    target_size=(img_row, img_col),
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "train", #duitai training file theke kintu
    x_col='filename',
    y_col='age',
    class_mode='categorical',
    target_size=(img_row, img_col),
    batch_size=batch_size
)


Found 21337 validated image filenames belonging to 5 classes.
Found 2371 validated image filenames belonging to 5 classes.


In [80]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "train", 
    x_col='filename',
    y_col='age',
    class_mode='categorical',                    #'multi_output'],
    target_size=(img_row, img_col),
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "train", #duitai training file theke kintu
    x_col='filename',
    y_col='age',
    class_mode='categorical',
    target_size=(img_row, img_col),
    batch_size=batch_size
)


Found 21337 validated image filenames belonging to 5 classes.
Found 2371 validated image filenames belonging to 5 classes.


In [96]:
model.compile(optimizer=Adam(lr=.1.600000018697756e-7),                              #{'out_1':'adam','out_2':'adam'}, 
              loss='categorical_crossentropy',
              metrics =['accuracy']                           #{'out_1':"accuracy",'out_2':"accuracy"}
             )

In [92]:
from keras.optimizers import RMSprop,Adam    #import activation function that you need
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau 



learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', #helps to monitor if val_loss;if it doesn't reduce after 5 epochs, the learning rate will be decreased and checked again in the same way;this will be done upto lr=.001
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks=[learning_rate_reduction]

In [93]:
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size)

Epoch 1/30
166/166 [==============================] - 60s 359ms/step - loss: 1.3480 - accuracy: 0.4319 - val_loss: 1.2646 - val_accuracy: 0.4549
Epoch 2/30
166/166 [==============================] - 57s 346ms/step - loss: 1.2759 - accuracy: 0.4548 - val_loss: 1.1228 - val_accuracy: 0.4974
Epoch 3/30
166/166 [==============================] - 37s 226ms/step - loss: 1.2561 - accuracy: 0.4694 - val_loss: 1.1189 - val_accuracy: 0.5060
Epoch 4/30
166/166 [==============================] - 38s 226ms/step - loss: 1.2504 - accuracy: 0.4630 - val_loss: 1.2447 - val_accuracy: 0.4887
Epoch 5/30
166/166 [==============================] - 38s 231ms/step - loss: 1.2349 - accuracy: 0.4659 - val_loss: 1.2294 - val_accuracy: 0.5188
Epoch 6/30
166/166 [==============================] - 38s 231ms/step - loss: 1.2396 - accuracy: 0.4674 - val_loss: 1.1690 - val_accuracy: 0.4991

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 7/30
166/166 [============================

In [101]:
model.save("age_prediction_50epochs")

In [99]:
checkpoint = ModelCheckpoint("age_prediction_50.h5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 15,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

In [100]:
history = model.fit_generator(
    train_generator,
    epochs=20,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size)

Epoch 1/20
166/166 [==============================] - 37s 221ms/step - loss: 1.1642 - accuracy: 0.5039 - val_loss: 1.1210 - val_accuracy: 0.5139

Epoch 00001: val_accuracy improved from -inf to 0.51389, saving model to age_prediction_50.h5
Epoch 2/20
166/166 [==============================] - 37s 223ms/step - loss: 1.1831 - accuracy: 0.4957 - val_loss: 1.1627 - val_accuracy: 0.5391

Epoch 00002: val_accuracy improved from 0.51389 to 0.53906, saving model to age_prediction_50.h5
Epoch 3/20
166/166 [==============================] - 38s 229ms/step - loss: 1.1738 - accuracy: 0.5020 - val_loss: 1.0617 - val_accuracy: 0.5041

Epoch 00003: val_accuracy did not improve from 0.53906
Epoch 4/20
166/166 [==============================] - 37s 225ms/step - loss: 1.1790 - accuracy: 0.4932 - val_loss: 1.0802 - val_accuracy: 0.5451

Epoch 00004: val_accuracy improved from 0.53906 to 0.54514, saving model to age_prediction_50.h5
Epoch 5/20
166/166 [==============================] - 38s 226ms/step - lo